# Kong: Performance Evaluation

In this notebook, we report on some experimental results obtained with Kong.

### Setup Analysis

Import librairies

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

Path to data

In [ ]:
path_data = 'OUTPUTS/'

Settings

In [ ]:
TIMEOUT = 3600

## Load Data

Reductions data frame.

In [ ]:
# Read 'reductions.csv'
df_reductions = pd.read_csv(path_data + 'reductions.csv')
df_reductions

Computations data frame.

In [ ]:
# Read 'computations.csv'
df_computations = pd.read_csv(path_data + 'computations.csv')
df_computations

## Tool Confidence rate

### Reliability

In [ ]:
reliability = df_computations.query('CORRECTNESS == False').shape[0] / df_computations.shape[0] * 100
reliability

### Correct Matrices

Correct matrices.

In [ ]:
correct_matrices = df_computations.query('CORRECTNESS == True and TIME_KONG == TIME_KONG').shape[0]
correct_matrices

### Summary Table

Computed matrices using Kong.

In [ ]:
computed_matrices_using_Kong = df_computations.query('TIME_KONG == TIME_KONG').shape[0]
computed_matrices_using_Kong

Computed matrices using caesar.

In [ ]:
computed_matrices_using_caesar = df_computations.query('TIME_CAESAR == TIME_CAESAR').shape[0]
computed_matrices_using_caesar

In [ ]:
summary = {'Reliability': [reliability, np.nan], 'Computed Matrices': [computed_matrices_using_Kong, computed_matrices_using_caesar]}
pd.DataFrame(data=summary, index=['Kong', 'Caesar'])

## Analysis

Performance evalutation of the *polyhedral abstraction* approach.

### Reduction Ratio 

Reduction ratio among instances.

In [ ]:
# Count instances with same reduction ratio
ratio_frequency = df_reductions['RATIO'].value_counts().sort_index(ascending=False)

# Cumulative data frame
df_ratio_frequency = pd.DataFrame({'INSTANCES': ratio_frequency.values}).cumsum()
df_ratio_frequency['RATIO'] = ratio_frequency.index

# Add row '0'
df_first_row = pd.DataFrame([{'INSTANCES': 1, 'RATIO': df_ratio_frequency['RATIO'][0]}])
df_ratio_frequency = pd.concat([df_first_row, df_ratio_frequency])

# Draw instances reduction ratio
df_ratio_frequency.plot.area(x='INSTANCES', xlim=(0,df_reductions.shape[0]), ylim=(0,100), color='cornflowerblue', figsize=(13,3), legend='')
plt.xlabel('Number of instances', fontsize=13)
plt.ylabel('Reduction ratio (%)', fontsize=13)

plt.savefig('reduction.png', bbox_inches = 'tight')

plt.show()

### General Performance Overview

In [ ]:
# Get computations with reduction ratio
df_computations_with_ratio = df_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE')

# Display mean reduction time
print('Mean reduction time: ', df_computations_with_ratio['TIME'].mean())

In [ ]:
def information_per_reduction_range(ratio_min, ratio_max):
    """ Return information for a given reduction range.
    """
    df = df_computations_with_ratio.query('RATIO >= {} and RATIO <= {}'.format(ratio_min, ratio_max))

    reduction_range = '{}-{}%'.format(ratio_min, ratio_max)

    number_instances = df[['INSTANCE']].drop_duplicates().shape[0]

    computed_matrices_using_Kong = df.query('TIME_KONG == TIME_KONG').shape[0]
    computed_matrices_using_caesar = df.query('TIME_CAESAR == TIME_CAESAR').shape[0]
    gain = (1 - computed_matrices_using_caesar / computed_matrices_using_Kong) * 100

    return [reduction_range, number_instances, computed_matrices_using_Kong, computed_matrices_using_caesar, gain]

Summary table.

In [ ]:
performance_overview = pd.DataFrame([information_per_reduction_range(ratio_min, ratio_max) for ratio_min, ratio_max in [[30, 100], [30, 70], [60, 99], [100,100]]], columns=['Reduction Ratio', 'Number of Instances', 'Computed Matrices using Kong', 'Computed Matrices using Caesar', 'Gain (%)'])
performance_overview.set_index('Reduction Ratio')

### Matrix Computation Times: With VS Without Reduction

Comparaison of the number of computed matrices in a limited time between Kong and caesar.

In [ ]:
def time_with_vs_without_reduction(ratio_min, ratio_max):
    """ Plot matrix computation times with vs without reduction.
    """
    # Get computed matrices for a given reduction range (remove isntances that timeout using Kong and caesar)
    df = df_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and (TIME_KONG == TIME_KONG or TIME_CAESAR == TIME_CAESAR)'.format(ratio_min, ratio_max))
    
    # Replace timeout NaN values by the timeout value
    df.loc[df.TIME_KONG != df.TIME_KONG, 'TIME_KONG'] = TIMEOUT
    df.loc[df.TIME_CAESAR != df.TIME_CAESAR, 'TIME_CAESAR'] = TIMEOUT

    # Replace 0 values by 0.1
    df.loc[df.TIME_KONG == 0, 'TIME_KONG'] = 0.01
    df.loc[df.TIME_CAESAR == 0, 'TIME_CAESAR'] = 0.01

    # Get times using Kong and caesar 
    x = df['TIME_CAESAR'].to_numpy()
    y = df['TIME_KONG'].to_numpy()
     
    # Plot time with vs without reduction with log scale
    plt.figure(figsize=(8,8))
    ax = plt.gca()
    plt.scatter(x=x, y=y, marker='+')  
    plt.plot(np.linspace(0.00, TIMEOUT), np.linspace(0.00, TIMEOUT), color='black', linestyle='--', lw=2, scalex=False, scaley=False)
    plt.plot(np.linspace(0.00, TIMEOUT), 0.1 * np.linspace(0.00, TIMEOUT), color='orange', linestyle=':', lw=2, scalex=False, scaley=False)
    plt.plot(np.linspace(0.00, TIMEOUT), 0.01 * np.linspace(0.00, TIMEOUT), color='red', linestyle=':', lw=2, scalex=False, scaley=False)
    ax.set_xscale('log')
    ax.set_yscale('log')
    plt.xlabel('Computation time without reduction (s)')
    plt.ylabel('Computation time with reduction (s)')
    plt.savefig("time_{}_{}.png".format(ratio_min, ratio_max), bbox_inches = 'tight')
    plt.show()

    # Plot number of properties, number of computed matrices with reduction and without reduction
    plt.figure(figsize=(1,8))
    plt.bar([0], [df_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {}'.format(ratio_min, ratio_max)).shape[0]], color='orange', label='All instances')
    plt.bar([0], [df_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and TIME_KONG < {}'.format(ratio_min, ratio_max, TIMEOUT)).shape[0]], label='Matrices computed with reduction')
    plt.bar([0], [df_computations.join(df_reductions.set_index('INSTANCE'), on='INSTANCE').query('RATIO >= {} and RATIO <= {} and TIME_CAESAR < {}'.format(ratio_min, ratio_max, TIMEOUT)).shape[0]], color='cyan', label='Matrices computed without reduction')
    plt.legend(loc='upper left')
    ax = plt.gca()
    ax.axes.xaxis.set_visible(False)
    plt.xlim([0, 0.1])
    plt.savefig("bar_{}_{}.png".format(ratio_min, ratio_max), bbox_inches = 'tight')
    plt.show()

In [ ]:
time_with_vs_without_reduction(50, 100)